<a href="https://colab.research.google.com/github/artamaral/OpticatOnline/blob/master/Opticat_ACES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import pandas as pd

#Need to generete curl before to run to update headers
1 logging

2 look for a specific pn

3 copy curl

In [ ]:
list_data = []

In [ ]:
def newItem(pn):
  headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'content-type': 'application/json',
    'origin': 'https://web.tecalliance.net',
    'priority': 'u=1, i',
    'referer': 'https://web.tecalliance.net/',
    'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
    'x-api-key': '2BeBXg6DZsG5EhoE8dy18zrf9ieWU3JSCtNYUXWaKcbsAapjghso',
    'x-catalog': 'BMW63A8UhKQ7KKNh5vRXs',
    'x-log-tracking-id': 'fbb2176c-67a0-4ef2-aeb6-f2d0a29147a6',
}

  json_data = {
    'getAutoCarePartApplications': {
        'brandCode': 'BBZH',
        'partNumber': pn,
        'includeResults': True,
        'includeMatchingBaseVehicles': True,
        'perPage': 100,
    },
}

  response = requests.post(
    'https://webservice.opticatonline.com/autocare/v1/services/Catalog.jsonEndpoint',
    headers=headers,
    json=json_data,
)
  return response.text

Open the file that have all partnumbers

In [ ]:
# prompt: open a csv file as a list

import csv

def open_csv_as_list(filename):
  """Opens a CSV file and returns its contents as a list of lists."""
  with open(filename, 'r') as csvfile:
    reader = csv.reader(csvfile)
    return list(reader)

# Example usage:
# my_list = open_csv_as_list('my_file.csv')
# print(my_list)


In [ ]:
csvData = open_csv_as_list('/content/LUK_opticat_pn.csv')

Make sure that in all the fields have data, other wise it wiil fail. So use try except.

In [ ]:
list_data = []
for pn in csvData:

  pn = str(pn).replace('[', '').replace(']', '').replace("'", '')
  print(pn)
  response = newItem(pn)
  data = json.loads(response)
  for i in range(len(data['results'])):

    try:
      Engine = data['results'][i]['acesApp']['attributes'][1]['attributeValueName']
    except:
      continue

    opticat_data = {
        "pn":data['results'][i]['acesApp']['partNumber'],
        "BVId":data['results'][i]['acesApp']['baseVehicleId'],
        "VehName":data['results'][i]['acesApp']['baseVehicleName'],
          "Engine":Engine,
        "PN_Desc":data['results'][i]['acesApp']['partTypeName']
  }
    list_data.append(opticat_data)



Save as csv

In [ ]:
list_data
allData = pd.DataFrame(list_data)

In [ ]:
allData

,pn,BVId,VehName,Engine,PN_Desc
0,07-233,129981,2015 Ford Fiesta,L4 1.6L 1596cc,Automatic Dual Clutch Transmission Clutch Kit
1,07-233,127938,2014 Ford Focus,L4 2.0L 1999cc,Automatic Dual Clutch Transmission Clutch Kit
2,07-233,147732,2019 Ford Fiesta,L4 1.6L 1596cc,Automatic Dual Clutch Transmission Clutch Kit
3,07-233,142882,2017 Ford Fiesta,L3 1.0L 999cc,Automatic Dual Clutch Transmission Clutch Kit
4,07-233,131027,2015 Ford Focus,L4 2.0L 1999cc,Automatic Dual Clutch Transmission Clutch Kit
...,...,...,...,...,...
11669,22-044,30491,2010 Volvo C70,N/A,Transmission Clutch Kit
11670,22-044,30174,2009 Volvo C70,N/A,Transmission Clutch Kit
11671,22-044,30173,2009 Volvo C30,N/A,Transmission Clutch Kit
11672,22-044,122240,2013 Volvo C30,L5 2.5L 2521cc,Transmission Clutch Kit


In [ ]:
# prompt: gerar um arquivo csv a partir de allData data frame

allData.to_csv('output.csv', index=False)
